# LLaMA 3.1 Anomaly - Hypothesis Tests

**Date:** 2026-01-05
**Goal:** Understand WHY LLaMA 3.1 contracts (0.48x) while Mistral expands (1.37x)

## Hypotheses to Test:

1. **Titanium Projector:** Vocab size 128k vs 32k → W_U must be massive
2. **Long-Context Dampening:** 128k context → systematic dampening needed
3. **Grokking:** 15T tokens training → ultra-efficient representations

In [ ]:
# Cell 0: HuggingFace Login (REQUIRED for gated models)
from huggingface_hub import login

# Option 1: Use your HF token directly
# login(token="hf_YOUR_TOKEN_HERE")

# Option 2: Interactive login (will prompt for token)
login()

print("HuggingFace login complete!")

In [ ]:
# Cell 1: Setup
import torch
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import matplotlib.pyplot as plt
import json
import os

# Results directory
RESULTS_DIR = './Results'
os.makedirs(RESULTS_DIR, exist_ok=True)

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
print("Setup complete")

In [ ]:
# Cell 2: Model Definitions for Comparison

MODELS_TO_COMPARE = {
    # LLaMA 3.1 (CONTRACTS - 0.48x)
    'llama3.1-8b': {
        'hf_name': 'meta-llama/Llama-3.1-8B',
        'vocab_size': 128256,
        'context_length': 128000,
        'training_tokens': '15T',
        'last_gain': 0.48,
        'behavior': 'CONTRACTS'
    },
    # Mistral (EXPANDS - 1.37x)
    'mistral-7b': {
        'hf_name': 'mistralai/Mistral-7B-v0.1',
        'vocab_size': 32000,
        'context_length': 8192,
        'training_tokens': '~7-8T',
        'last_gain': 1.37,
        'behavior': 'EXPANDS'
    },
    # Gemma 2 9B (ungated alternative to Gemma 7B)
    'gemma2-9b': {
        'hf_name': 'google/gemma-2-9b',
        'vocab_size': 256000,
        'context_length': 8192,
        'training_tokens': '8T',
        'last_gain': 2.85,  # Using Gemma 7B reference
        'behavior': 'EXPANDS'
    }
}

print("Models defined:")
for name, info in MODELS_TO_COMPARE.items():
    print(f"  {name}: vocab={info['vocab_size']:,}, ctx={info['context_length']:,}, {info['behavior']}")

In [ ]:
# Cell 3: Hypothesis 1 - Titanium Projector Test
# Measure W_U (unembedding/lm_head) spectral norm

def analyze_unembedding_matrix(model_name, hf_name):
    """Analyze the unembedding matrix (lm_head) of a model."""
    print(f"\n{'='*60}")
    print(f"Analyzing: {model_name}")
    print(f"{'='*60}")
    
    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        hf_name,
        torch_dtype=torch.float16,
        device_map='auto',
        trust_remote_code=True
    )
    
    # Get lm_head (unembedding matrix)
    lm_head = model.lm_head.weight.data.float()  # [vocab_size, hidden_dim]
    
    results = {
        'model': model_name,
        'shape': list(lm_head.shape),
        'vocab_size': lm_head.shape[0],
        'hidden_dim': lm_head.shape[1],
    }
    
    # Compute norms
    print(f"\nW_U shape: {lm_head.shape}")
    
    # 1. Frobenius norm (total energy)
    frob_norm = torch.linalg.norm(lm_head, ord='fro').item()
    results['frobenius_norm'] = frob_norm
    print(f"Frobenius norm: {frob_norm:.2f}")
    
    # 2. Spectral norm (largest singular value) - THE KEY METRIC
    # This tells us the maximum amplification factor
    spectral_norm = torch.linalg.norm(lm_head, ord=2).item()
    results['spectral_norm'] = spectral_norm
    print(f"Spectral norm (σ_max): {spectral_norm:.2f}")
    
    # 3. Mean row norm (average token representation magnitude)
    row_norms = torch.linalg.norm(lm_head, dim=1)
    mean_row_norm = row_norms.mean().item()
    std_row_norm = row_norms.std().item()
    results['mean_row_norm'] = mean_row_norm
    results['std_row_norm'] = std_row_norm
    print(f"Mean row norm: {mean_row_norm:.4f} +/- {std_row_norm:.4f}")
    
    # 4. Normalized spectral norm (per vocab token)
    norm_per_vocab = spectral_norm / np.sqrt(lm_head.shape[0])
    results['spectral_per_sqrt_vocab'] = norm_per_vocab
    print(f"Spectral / sqrt(vocab): {norm_per_vocab:.4f}")
    
    # 5. Effective amplification (compensating for residual contraction)
    last_gain = MODELS_TO_COMPARE[model_name]['last_gain']
    effective_amp = spectral_norm * last_gain
    results['effective_amplification'] = effective_amp
    results['last_gain'] = last_gain
    print(f"\nEffective amplification (sigma_max x last_gain):")
    print(f"  {spectral_norm:.2f} x {last_gain:.2f} = {effective_amp:.2f}")
    
    # Clean up
    del model
    torch.cuda.empty_cache()
    
    return results

print("Analysis function defined")

In [ ]:
# Cell 4: Run Hypothesis 1 Test - LLaMA 3.1 ONLY
# (Run this separately to avoid memory issues)

all_results = {}

# LLaMA 3.1 first (most important)
model_name = 'llama3.1-8b'
model_info = MODELS_TO_COMPARE[model_name]

try:
    results = analyze_unembedding_matrix(model_name, model_info['hf_name'])
    all_results[model_name] = results
except Exception as e:
    print(f"Error loading {model_name}: {e}")
    all_results[model_name] = {'error': str(e)}

print("\nLLaMA 3.1 analysis complete!")

In [ ]:
# Cell 5: Run Hypothesis 1 Test - Mistral

model_name = 'mistral-7b'
model_info = MODELS_TO_COMPARE[model_name]

try:
    results = analyze_unembedding_matrix(model_name, model_info['hf_name'])
    all_results[model_name] = results
except Exception as e:
    print(f"Error loading {model_name}: {e}")
    all_results[model_name] = {'error': str(e)}

print("\nMistral analysis complete!")

In [ ]:
# Cell 6: Run Hypothesis 1 Test - Gemma 2 (optional)

model_name = 'gemma2-9b'
model_info = MODELS_TO_COMPARE[model_name]

try:
    results = analyze_unembedding_matrix(model_name, model_info['hf_name'])
    all_results[model_name] = results
except Exception as e:
    print(f"Error loading {model_name}: {e}")
    all_results[model_name] = {'error': str(e)}

print("\nGemma 2 analysis complete!")

In [ ]:
# Cell 7: Compare Results

print("\n" + "="*70)
print("TITANIUM PROJECTOR HYPOTHESIS - RESULTS")
print("="*70)

print("\n| Model | Vocab | sigma_max (W_U) | Last Gain | Effective Amp |")
print("|-------|-------|-----------------|-----------|---------------|")

for name, r in all_results.items():
    if 'error' not in r:
        vocab = r['vocab_size']
        sigma = r['spectral_norm']
        gain = r['last_gain']
        eff = r['effective_amplification']
        print(f"| {name:13} | {vocab:6,} | {sigma:15.2f} | {gain:9.2f} | {eff:13.2f} |")

print("\n" + "-"*70)
print("HYPOTHESIS EVALUATION:")
print("-"*70)

# Check if LLaMA's W_U compensates for its contraction
llama_data = all_results.get('llama3.1-8b', {})
mistral_data = all_results.get('mistral-7b', {})

if 'error' not in llama_data and 'error' not in mistral_data:
    llama_eff = llama_data.get('effective_amplification', 0)
    mistral_eff = mistral_data.get('effective_amplification', 0)
    
    llama_sigma = llama_data.get('spectral_norm', 0)
    mistral_sigma = mistral_data.get('spectral_norm', 0)
    
    sigma_ratio = llama_sigma / mistral_sigma if mistral_sigma > 0 else 0
    eff_ratio = llama_eff / mistral_eff if mistral_eff > 0 else 0
    
    print(f"\nLLaMA sigma_max: {llama_sigma:.2f}")
    print(f"Mistral sigma_max: {mistral_sigma:.2f}")
    print(f"Sigma ratio (LLaMA/Mistral): {sigma_ratio:.2f}x")
    
    print(f"\nLLaMA effective amp: {llama_eff:.2f}")
    print(f"Mistral effective amp: {mistral_eff:.2f}")
    print(f"Effective ratio (LLaMA/Mistral): {eff_ratio:.2f}x")
    
    print("\n" + "="*70)
    if 0.7 < eff_ratio < 1.3:
        print("VERDICT: HYPOTHESIS SUPPORTED!")
        print("="*70)
        print("\nLLaMA's larger W_U compensates for its residual contraction.")
        print("The 'broadcast' happens via Static Amplification, not stream expansion.")
    elif eff_ratio > 1.3:
        print("VERDICT: HYPOTHESIS EXCEEDED!")
        print("="*70)
        print(f"\nLLaMA actually has {eff_ratio:.1f}x MORE effective amplification.")
        print("The W_U OVER-compensates for the residual contraction!")
    else:
        print("VERDICT: HYPOTHESIS PARTIALLY SUPPORTED")
        print("="*70)
        print(f"\nLLaMA has {eff_ratio:.1f}x the effective amplification.")
        print(f"W_U is {sigma_ratio:.1f}x larger but doesn't fully compensate.")
        print("Other factors may contribute.")
else:
    print("Cannot evaluate - missing model data")

In [ ]:
# Cell 8: Visualization

# Filter out models with errors
valid_models = [m for m in all_results.keys() if 'error' not in all_results[m]]

if len(valid_models) >= 2:
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    colors = ['#e74c3c' if 'llama' in m else '#3498db' if 'mistral' in m else '#2ecc71' for m in valid_models]

    # Plot 1: Spectral Norm
    ax1 = axes[0]
    spectral_norms = [all_results[m].get('spectral_norm', 0) for m in valid_models]
    bars1 = ax1.bar(valid_models, spectral_norms, color=colors)
    ax1.set_ylabel('Spectral Norm (sigma_max)')
    ax1.set_title('W_U Spectral Norm\n(Maximum Amplification Factor)')
    ax1.tick_params(axis='x', rotation=45)
    for bar, val in zip(bars1, spectral_norms):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                 f'{val:.1f}', ha='center', va='bottom')

    # Plot 2: Last Layer Gain
    ax2 = axes[1]
    last_gains = [all_results[m].get('last_gain', 0) for m in valid_models]
    bars2 = ax2.bar(valid_models, last_gains, color=colors)
    ax2.set_ylabel('Last Layer Gain')
    ax2.set_title('Residual Stream\nLast Layer Gain')
    ax2.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5)
    ax2.tick_params(axis='x', rotation=45)
    for bar, val in zip(bars2, last_gains):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05, 
                 f'{val:.2f}x', ha='center', va='bottom')

    # Plot 3: Effective Amplification
    ax3 = axes[2]
    effective_amps = [all_results[m].get('effective_amplification', 0) for m in valid_models]
    bars3 = ax3.bar(valid_models, effective_amps, color=colors)
    ax3.set_ylabel('Effective Amplification')
    ax3.set_title('Effective Amplification\n(sigma_max x Last Gain)')
    ax3.tick_params(axis='x', rotation=45)
    for bar, val in zip(bars3, effective_amps):
        ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                 f'{val:.1f}', ha='center', va='bottom')

    plt.tight_layout()
    plt.savefig(f'{RESULTS_DIR}/titanium_projector_hypothesis.png', dpi=150, bbox_inches='tight')
    plt.show()

    print(f"\nVisualization saved to {RESULTS_DIR}/titanium_projector_hypothesis.png")
else:
    print("Not enough valid models for visualization")

In [ ]:
# Cell 9: Hypothesis 2 - RoPE Theta Analysis
# Check RoPE base frequency differences

print("\n" + "="*70)
print("HYPOTHESIS 2: LONG-CONTEXT DAMPENING (RoPE Analysis)")
print("="*70)

rope_analysis = {}

for model_name, model_info in MODELS_TO_COMPARE.items():
    try:
        config = AutoConfig.from_pretrained(model_info['hf_name'], trust_remote_code=True)
        
        # Extract RoPE parameters
        rope_theta = getattr(config, 'rope_theta', None)
        rope_scaling = getattr(config, 'rope_scaling', None)
        max_position = getattr(config, 'max_position_embeddings', None)
        
        rope_analysis[model_name] = {
            'rope_theta': rope_theta,
            'rope_scaling': rope_scaling,
            'max_position': max_position,
            'context_length': model_info['context_length']
        }
        
        print(f"\n{model_name}:")
        print(f"  RoPE theta: {rope_theta:,}" if rope_theta else "  RoPE theta: None")
        print(f"  RoPE scaling: {rope_scaling}")
        print(f"  Max positions: {max_position:,}" if max_position else "  Max positions: None")
        print(f"  Target context: {model_info['context_length']:,}")
        
    except Exception as e:
        print(f"Error for {model_name}: {e}")

print("\n" + "-"*70)
print("HYPOTHESIS 2 EVALUATION:")
print("-"*70)

llama_theta = rope_analysis.get('llama3.1-8b', {}).get('rope_theta', 0)
mistral_theta = rope_analysis.get('mistral-7b', {}).get('rope_theta', 0)

if llama_theta and mistral_theta:
    ratio = llama_theta / mistral_theta
    print(f"\nLLaMA RoPE theta: {llama_theta:,}")
    print(f"Mistral RoPE theta: {mistral_theta:,}")
    print(f"Ratio: {ratio:.1f}x")
    
    print("\n" + "="*70)
    if ratio > 10:
        print("VERDICT: HYPOTHESIS SUPPORTED!")
        print("="*70)
        print("\nLLaMA uses much larger RoPE theta for long-context stability.")
        print("This may require dampened residual streams to prevent explosion.")
    else:
        print("VERDICT: INCONCLUSIVE")
        print("="*70)
        print("\nRoPE theta difference alone may not explain the behavior.")
else:
    print("Cannot evaluate - missing RoPE theta data")

In [ ]:
# Cell 10: Save All Results

final_results = {
    'experiment': 'LLaMA 3.1 Anomaly - Hypothesis Tests',
    'date': '2026-01-05',
    'hypothesis_1_titanium_projector': {
        'description': 'W_U norm compensates for residual contraction',
        'results': all_results
    },
    'hypothesis_2_rope': {
        'description': 'Long-context RoPE requires dampening',
        'results': rope_analysis
    },
    'hypothesis_3_grokking': {
        'description': 'Training scale causes efficient representations',
        'status': 'THEORETICAL - Requires checkpoint analysis'
    }
}

output_path = f'{RESULTS_DIR}/llama_anomaly_hypothesis_tests.json'
with open(output_path, 'w') as f:
    json.dump(final_results, f, indent=2, default=str)

print(f"Results saved to {output_path}")
print("\n" + "="*70)
print("EXPERIMENT COMPLETE")
print("="*70)

In [ ]:
# Cell 11: DOWNLOAD ALL RESULTS
# Run this cell to download all files to your computer

from google.colab import files
import glob

print("="*70)
print("DOWNLOADING RESULTS...")
print("="*70)

# Download JSON
json_path = f'{RESULTS_DIR}/llama_anomaly_hypothesis_tests.json'
if os.path.exists(json_path):
    print(f"\nDownloading: {json_path}")
    files.download(json_path)

# Download PNG
png_path = f'{RESULTS_DIR}/titanium_projector_hypothesis.png'
if os.path.exists(png_path):
    print(f"Downloading: {png_path}")
    files.download(png_path)

print("\n" + "="*70)
print("DOWNLOAD COMPLETE!")
print("="*70)

## Summary

### Hypothesis 1: Titanium Projector
- **Test:** Compare W_U spectral norms
- **Prediction:** LLaMA's sigma_max should be ~2-3x larger to compensate for 0.48x contraction

### Hypothesis 2: Long-Context Dampening
- **Test:** Compare RoPE theta values
- **Prediction:** LLaMA should have much larger RoPE theta for 128k context

### Hypothesis 3: Grokking
- **Status:** THEORETICAL
- **Test would require:** Training checkpoints to measure gain over training time
- **Prediction:** Gain should collapse at the "grokking point"